In [2]:
!pip install tensorflow

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.23.5 which is incompatible.



     -------------------------------------- 272.8/272.8 MB 5.0 MB/s eta 0:00:00
     -------------------------------------- 126.5/126.5 kB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 10.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 24.4/24.4 MB 7.9 MB/s eta 0:00:00
     ---------------------------------------- 14.7/14.7 MB 9.5 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB 3.5 MB/s eta 0:00:00
     -------------------------------------- 422.5/422.5 kB 6.5 MB/s eta 0:00:00
     -------------------------

In [10]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
# from wordcloud import WordCloud

DATA_PATH = './data/' #데이터경로 설정
print('파일 크기: ')
for file in os.listdir(DATA_PATH):
  if 'txt' in file:
    print(file.ljust(30)+str(round(os.path.getsize(DATA_PATH+ file) / 100000,2))+'MB')

파일 크기: 
ratings.txt                   195.15MB
ratings_test.txt              48.93MB
ratings_train.txt             146.29MB


In [ ]:
# 참고 : https://wonhwa.tistory.com/35

In [11]:
train_data = pd.read_csv(DATA_PATH + 'ratings_train.txt',header = 0, delimiter = '\t', quoting=3)
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [12]:
# 문자열 아닌 데이터 모두 제거
train_review = [review for review in train_data['document'] if type(review) is str]

In [13]:
import numpy as np
import pandas as pd
import re
import json
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [15]:
DATA_PATH = './data/' # 데이터 경로 설정
train_data = pd.read_csv(DATA_PATH+'ratings_train.txt', header = 0, delimiter='\t', quoting=3)

train_data['document'][:5]

0                                  아 더빙.. 진짜 짜증나네요 목소리
1                    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                        교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
Name: document, dtype: object

In [16]:
#전처리 함수 만들기
def preprocessing(review, okt, remove_stopwords = False, stop_words =[]):
  #함수인자설명
  # review: 전처리할 텍스트
  # okt: okt객체를 반복적으로 생성하지 않고 미리 생성 후 인자로 받음
  # remove_stopword: 불용어를 제거할지 여부 선택. 기본값 False
  # stop_words: 불용어 사전은 사용자가 직접 입력, 기본값 빈 리스트

  # 1. 한글 및 공백 제외한 문자 모두 제거
  review_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',review)
  
  #2. okt 객체를 활용하여 형태소 단어로 나눔
  word_review = okt.morphs(review_text,stem=True)

  if remove_stopwords:
    #3. 불용어 제거(선택)
    word_review = [token for token in word_review if not token in stop_words]
  return word_review

In [17]:
# 전체 텍스트 전처리
stop_words = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한']
okt = Okt()
clean_train_review = []

for review in train_data['document']:
  # 리뷰가 문자열인 경우만 전처리 진행
  if type(review) == str:
    clean_train_review.append(preprocessing(review,okt,remove_stopwords=True,stop_words= stop_words))
  else:
    clean_train_review.append([]) #str이 아닌 행은 빈칸으로 놔두기

clean_train_review[:4]

[['더빙', '진짜', '짜증나다', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'],
 ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '다'],
 ['교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정']]

In [18]:
#테스트 리뷰도 동일하게 전처리
test_data = pd.read_csv(DATA_PATH + 'ratings_test.txt', header = 0, delimiter='\t', quoting=3)

clean_test_review = []
for review in test_data['document']:
  if type(review) == str:
    clean_test_review.append(preprocessing(review, okt, remove_stopwords=True, stop_words=stop_words))
  else:
    clean_test_review.append([])

In [19]:
# 인덱스 벡터 변환 후 일정 길이 넘어가거나 모자라는 리뷰 패딩처리
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_review)
train_sequences = tokenizer.texts_to_sequences(clean_train_review)
test_sequences = tokenizer.texts_to_sequences(clean_test_review)

word_vocab = tokenizer.word_index #단어사전형태
MAX_SEQUENCE_LENGTH = 8 #문장 최대 길이

#학습 데이터
train_inputs = pad_sequences(train_sequences,maxlen=MAX_SEQUENCE_LENGTH,padding='post')

#학습 데이터 라벨 벡터화
train_labels = np.array(train_data['label'])

#평가 데이터 
test_inputs = pad_sequences(test_sequences,maxlen=MAX_SEQUENCE_LENGTH,padding='post')
#평가 데이터 라벨 벡터화
test_labels = np.array(test_data['label'])

In [20]:
DEFAULT_PATH  = './data/' # 경로지정
DATA_PATH = 'CLEAN_DATA/' #.npy파일 저장 경로지정
TRAIN_INPUT_DATA = 'nsmc_train_input.npy'
TRAIN_LABEL_DATA = 'nsmc_train_label.npy'
TEST_INPUT_DATA = 'nsmc_test_input.npy'
TEST_LABEL_DATA = 'nsmc_test_label.npy'
DATA_CONFIGS = 'data_configs.json'

data_configs={}
data_configs['vocab'] = word_vocab
data_configs['vocab_size'] = len(word_vocab) + 1

#전처리한 데이터들 파일로저장
import os

if not os.path.exists(DEFAULT_PATH + DATA_PATH):
  os.makedirs(DEFAULT_PATH+DATA_PATH)

#전처리 학습데이터 넘파이로 저장
np.save(open(DEFAULT_PATH+DATA_PATH+TRAIN_INPUT_DATA,'wb'),train_inputs)
np.save(open(DEFAULT_PATH+DATA_PATH+TRAIN_LABEL_DATA,'wb'),train_labels)
#전처리 테스트데이터 넘파이로 저장
np.save(open(DEFAULT_PATH+DATA_PATH+TEST_INPUT_DATA,'wb'),test_inputs)
np.save(open(DEFAULT_PATH+DATA_PATH+TEST_LABEL_DATA,'wb'),test_labels)

#데이터 사전 json으로 저장
json.dump(data_configs,open(DEFAULT_PATH + DATA_PATH + DATA_CONFIGS,'w'),ensure_ascii=False)

In [22]:
# 학습 데이터 불러오기
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
from tqdm import tqdm

#전처리 데이터 불러오기
DATA_PATH = './data/CLEAN_DATA/'
DATA_OUT = './data/DATA_OUT/'
INPUT_TRAIN_DATA = 'nsmc_train_input.npy'
LABEL_TRAIN_DATA = 'nsmc_train_label.npy'
DATA_CONFIGS = 'data_configs.json'

train_input = np.load(open(DATA_PATH + INPUT_TRAIN_DATA,'rb'))
train_input = pad_sequences(train_input,maxlen=train_input.shape[1])
train_label = np.load(open(DATA_PATH + LABEL_TRAIN_DATA,'rb'))
prepro_configs = json.load(open(DATA_PATH+DATA_CONFIGS,'r'))

In [23]:
model_name= 'cnn_classifier_kr'
BATCH_SIZE = 512
NUM_EPOCHS = 10
VALID_SPLIT = 0.1
MAX_LEN = train_input.shape[1]

kargs={'model_name': model_name, 'vocab_size':prepro_configs['vocab_size'],'embbeding_size':128, 'num_filters':100,'dropout_rate':0.5, 'hidden_dimension':250,'output_dimension':1}

In [24]:
class CNNClassifier(tf.keras.Model):

  def __init__(self, **kargs):
    super(CNNClassifier, self).__init__(name=kargs['model_name'])
    self.embedding = layers.Embedding(input_dim=kargs['vocab_size'], output_dim=kargs['embbeding_size'])
    self.conv_list = [layers.Conv1D(filters=kargs['num_filters'], kernel_size=kernel_size, padding='valid',activation = tf.keras.activations.relu,
                                    kernel_constraint = tf.keras.constraints.MaxNorm(max_value=3)) for kernel_size in [3,4,5]]
    self.pooling = layers.GlobalMaxPooling1D()
    self.dropout = layers.Dropout(kargs['dropout_rate'])
    self.fc1 = layers.Dense(units=kargs['hidden_dimension'],
                            activation = tf.keras.activations.relu,
                            kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))
    self.fc2 = layers.Dense(units=kargs['output_dimension'],
                            activation=tf.keras.activations.sigmoid,
                            kernel_constraint= tf.keras.constraints.MaxNorm(max_value=3.))
    

  def call(self,x):
    x = self.embedding(x)
    x = self.dropout(x)
    x = tf.concat([self.pooling(conv(x)) for conv in self.conv_list], axis = 1)
    x = self.fc1(x)
    x = self.fc2(x)
    return x

In [25]:
from tensorflow.keras.models import save_model

model = CNNClassifier(**kargs)
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics = [tf.keras.metrics.BinaryAccuracy(name='accuracy')])

#검증 정확도를 통한 EarlyStopping 기능 및 모델 저장 방식 지정
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=2)
checkpoint_path = DATA_OUT + model_name +'\weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)

if os.path.exists(checkpoint_dir):
  print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
  os.makedirs(checkpoint_dir, exist_ok=True)
  print("{} -- Folder create complete \n".format(checkpoint_dir))

cp_callback = ModelCheckpoint(
    checkpoint_path, monitor = 'val_accuracy', verbose=1, save_best_only = True,
    save_weights_only=True
)

history = model.fit(train_input, train_label, batch_size=BATCH_SIZE, epochs = NUM_EPOCHS,
                    validation_split=VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])

./data/DATA_OUT/cnn_classifier_kr -- Folder create complete 

Epoch 1/10
264/264 [==============================] - ETA: 0s - loss: 0.4596 - accuracy: 0.7738
Epoch 1: val_accuracy improved from -inf to 0.81993, saving model to ./data/DATA_OUT/cnn_classifier_kr\weights.h5
264/264 [==============================] - 25s 87ms/step - loss: 0.4596 - accuracy: 0.7738 - val_loss: 0.3901 - val_accuracy: 0.8199
Epoch 2/10
264/264 [==============================] - ETA: 0s - loss: 0.3516 - accuracy: 0.8459
Epoch 2: val_accuracy improved from 0.81993 to 0.82387, saving model to ./data/DATA_OUT/cnn_classifier_kr\weights.h5
264/264 [==============================] - 21s 81ms/step - loss: 0.3516 - accuracy: 0.8459 - val_loss: 0.3850 - val_accuracy: 0.8239
Epoch 3/10
264/264 [==============================] - ETA: 0s - loss: 0.3001 - accuracy: 0.8731
Epoch 3: val_accuracy improved from 0.82387 to 0.82793, saving model to ./data/DATA_OUT/cnn_classifier_kr\weights.h5
264/264 [===========================

In [27]:
INPUT_TEST_DATA = 'nsmc_test_input.npy'
LABEL_TEST_DATA = 'nsmc_test_label.npy'
SAVE_FILE_NM = 'weights.h5'

test_input = np.load(open(DATA_PATH+INPUT_TEST_DATA,'rb'))
test_input = pad_sequences(test_input,maxlen=test_input.shape[1])
test_label_data = np.load(open(DATA_PATH + LABEL_TEST_DATA, 'rb'))

model.load_weights('./data/DATA_OUT/cnn_classifier_kr/weights.h5')
model.evaluate(test_input, test_label_data)

1563/1563 [==============================] - 5s 3ms/step - loss: 0.3945 - accuracy: 0.8285


[0.39453819394111633, 0.828499972820282]

In [45]:
import numpy as np
import pandas as pd
import re
import json
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
okt = Okt()
tokenizer  = Tokenizer()

DATA_CONFIGS = 'data_configs.json'
prepro_configs = json.load(open('./data/CLEAN_DATA/'+DATA_CONFIGS,'r'))
prepro_configs['vocab'] = word_vocab

tokenizer.fit_on_texts(word_vocab)

MAX_LENGTH = 8 #문장최대길이

sentence = input('감성분석할 문장을 입력해 주세요.: ')
sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\\s ]','', sentence)
stopwords = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한'] # 불용어 추가할 것이 있으면 이곳에 추가
sentence = okt.morphs(sentence, stem=True) # 토큰화
sentence = [word for word in sentence if not word in stopwords] # 불용어 제거
vector  = tokenizer.texts_to_sequences(sentence)
pad_new = pad_sequences(vector, maxlen = MAX_LENGTH) # 패딩

model.load_weights('./data/DATA_OUT/cnn_classifier_kr/weights.h5') #모델 불러오기
predictions = model.predict(pad_new)
print("(1)",predictions)
print("(2)",predictions.squeeze(-1))
predictions = float(predictions.squeeze(-1)[1])
print("(3)",predictions)

if(predictions > 0.5):
  print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(predictions * 100))
else:
  print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - predictions) * 100))

감성분석할 문장을 입력해 주세요.: 향은 좀 강한 파우더향이구요 딱히 호불호를 말할 수가 없네요 그냥 달팡이라는 브렌드 믿고 쓰고있습니다
1/1 [==============================] - 0s 47ms/step
(1) [[0.47982526]
 [0.2628401 ]
 [0.6493049 ]
 [0.3330859 ]
 [0.47982526]
 [0.508738  ]
 [0.61330134]
 [0.08864752]
 [0.6115114 ]
 [0.5212156 ]
 [0.5445938 ]
 [0.5054894 ]
 [0.54759043]
 [0.1759883 ]
 [0.4829546 ]
 [0.5241169 ]
 [0.56501466]
 [0.5387089 ]
 [0.6381023 ]
 [0.6308482 ]
 [0.5421619 ]]
(2) [0.47982526 0.2628401  0.6493049  0.3330859  0.47982526 0.508738
 0.61330134 0.08864752 0.6115114  0.5212156  0.5445938  0.5054894
 0.54759043 0.1759883  0.4829546  0.5241169  0.56501466 0.5387089
 0.6381023  0.6308482  0.5421619 ]
(3) 0.2628400921821594
73.72% 확률로 부정 리뷰입니다.



In [67]:
no_evi = pd.read_excel("./csv/NoEvinue.xlsx")

In [77]:
evi = pd.read_excel("./csv/Evinue.xlsx")

In [78]:
# test = no_evi["리뷰 내용"].values.tolist()
test = evi["리뷰 내용"].values.tolist()
# positive = []
# negative = []
senti_prob = []
for review in test[300:600]:
    
#     sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\\s ]','', review)
    stopwords = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한'] # 불용어 추가할 것이 있으면 이곳에 추가
    sentence = okt.morphs(review, stem=True) # 토큰화
    sentence = [word for word in sentence if not word in stopwords] # 불용어 제거
    vector  = tokenizer.texts_to_sequences(sentence)
    pad_new = pad_sequences(vector, maxlen = 15) # 패딩

    model.load_weights('./data/DATA_OUT/cnn_classifier_kr/weights.h5') #모델 불러오기
    predictions = model.predict(pad_new)
#     print("(1)",predictions)
#     print("(2)",predictions.squeeze(-1))
    if len(predictions.squeeze(-1)) == 1:
        predictions = float(predictions.squeeze(-1)[0])
    else:
        predictions = float(predictions.squeeze(-1)[1])
#     print("(3)",predictions)
#     print()
#     print()
    senti_prob.append(predictions)
#     if(predictions > 0.5):
#         positive.append([test.index(review), predictions])
#         print(test.index(review),"{:.2f}% 확률로 긍정 리뷰입니다.\n".format(predictions * 100))
#     else:
#         negative.append([test.index(review), predictions])
#         print(test.index(review),"{:.2f}% 확률로 부정 리뷰입니다.\n".format((1-predictions) * 100))

2/2 [==============================] - 0s 12ms/step


5/5 [==============================] - 0s 7ms/step


In [36]:
print(len(positive))
print(len(negative))
print(len(positive)+len(negative))

102

In [54]:
test[403:]

17222

In [57]:
for review in test[404:]:
    print(review)
#     sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\\s ]','', review)
    stopwords = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한'] # 불용어 추가할 것이 있으면 이곳에 추가
    sentence = okt.morphs(review, stem=True) # 토큰화
    sentence = [word for word in sentence if not word in stopwords] # 불용어 제거
    vector  = tokenizer.texts_to_sequences(sentence)
    pad_new = pad_sequences(vector, maxlen = 15) # 패딩

    model.load_weights('./data/DATA_OUT/cnn_classifier_kr/weights.h5') #모델 불러오기
    predictions = model.predict(pad_new)
#     print("(1)",predictions)
#     print("(2)",predictions.squeeze(-1))
    if len(predictions.squeeze(-1)) == 1:
        predictions = float(predictions.squeeze(-1)[0])
    else:
        predictions = float(predictions.squeeze(-1)[1])
#     print("(3)",predictions)
#     print()
#     print()

    if(predictions > 0.5):
        positive.append([test.index(review), predictions])
        print(test.index(review),"{:.2f}% 확률로 긍정 리뷰입니다.\n".format(predictions * 100))
    else:
        negative.append([test.index(review), predictions])
        print(test.index(review),"{:.2f}% 확률로 부정 리뷰입니다.\n".format((1-predictions) * 100))

너무너무너무너무기름져서 못쓰겠어요
1/1 [==============================] - 0s 31ms/step
404 57.64% 확률로 긍정 리뷰입니다.

맘에 드는데 팩트가 고장이 났어요 꽉 안닫혀요
1/1 [==============================] - 0s 39ms/step
405 52.81% 확률로 긍정 리뷰입니다.

가격내림
1/1 [==============================] - 0s 45ms/step
406 52.98% 확률로 긍정 리뷰입니다.

좋아요
1/1 [==============================] - 0s 54ms/step
407 91.33% 확률로 긍정 리뷰입니다.

커버력도 없고 피부보정 1도 안되고 케이스는 너무 커서 휴대용으로 불편
1/1 [==============================] - 0s 32ms/step
408 54.86% 확률로 긍정 리뷰입니다.

누가 여러번 반품했나요? 포장 찌그러지고 때타고 매장에서 사는거 였다면 구매 안할정도로 포장이 지져분 동생 선물 주려고 했는데 그냥 내가 써야 겠네요
1/1 [==============================] - 0s 34ms/step
409 53.76% 확률로 긍정 리뷰입니다.

골든박스보다 지금이 싼이유가 뭡니까 골든박스땬 12000원넘고 지금은 11500원인데 얼마 차이안나는대 묘하게 기분 나쁘네
1/1 [==============================] - 0s 48ms/step
410 52.58% 확률로 긍정 리뷰입니다.

토요일에 급하게 썬크림 필요해서 금요일 오후 도착상품으로주문했는데 일요일 오후에 도착했네요로켓와우 의미없네요
1/1 [==============================] - 0s 24ms/step
411 52.81% 확률로 긍정 리뷰입니다.

배달이 안오네요 어디다가 물어야 하나요
1/1 [==============================] - 0s 41ms

11/11 [==============================] - 0s 5ms/step
439 69.04% 확률로 부정 리뷰입니다.

구매동기골프를 좋아하는 중년 여성입니다 골프가 참 재미있고 즐거운 운동이긴 한데 피부 노화 특히 자외선으로 인한 기미와 주름은 피할 수 없는 스포츠 인것 같아요 선크림을 덕지덕지 바르다보면 화장도 밀리고 눈에 들어가면 나무 따갑기도 하고 해서 워터프루프이면서 자외선 차단 효과가 탁월한 제품을 고르게 되었어요 골프여심 박세리님이 광고하는 제품이라 더욱더 믿음이 가서 구매하게 되었습니다 제품특징AHC 에어 리치 선스틱 은 물과 땀에 쉽게 지워지지 않고 오래도록 자외선 차단 효과를 유지해주는 강력한 워터프루프 효과의 롱래스팅 선스틱이다 칼라민 파우더가 함유돼 세범 컨트롤에 도움을 주며 가볍게 밀착되는 에어 텍스처로 보송한 마무리감을 선사한다 산뜻하고 빈틈없이 밀착돼 뛰어난 자외선 차단 효과를 선사하고 24시간 지속되는 피부 보습 효과 민감한 피부도 안심하고 사용할 수 있는 마일드한 처방까지 갖췄다 자외선 적외선 초미세먼지로부터 피부를 보호하고 물과 땀 모래 액티비티 마스크에도 쉽게 지워지지 않는다 들고 다니며 덧바르기도 편한 스틱형이다 사용후기생각보다 크리미해서 얼굴에 잘 발려요 화장도 밀리지 않게끔 얼굴에 잘 스며들고 번들번들한 느낌도 없네요 운동하고 땀 흘려도 눈에 들어가거나 하지 않아서 눈이 따갑거나 하는 증상이 적은것 같습니다 디자인도 이쁘고 블랙색상이아 심플하면서 세련된 느낌이 들어요 밑에 부분을 돌려서 올리는 타입이라 휴대하면서 굳이 손으로 바르지 않아도 얼굴에 밀면서 사용이 가능하기 때문에 정말 쉽고 간편하게 사용 할 수 있어서 매우 좋습니다 앞으로 열심히 쓰고 재구매 할 것 같아요제 최종점수는요  제후기가 도움이 되셨다면 도움이돼요 클릭 부탁드립니다
9/9 [==============================] - 0s 5ms/step
440 65.06% 확률로 긍정 리뷰입니다.

자외선차단제 구입할 때마다 

1/1 [==============================] - 0s 37ms/step
465 53.02% 확률로 긍정 리뷰입니다.

유분기 없이 뽀송해서 좋았어요 번들거리지도 않고 보송보송해서 지성에 좋은것같아요
1/1 [==============================] - 0s 35ms/step
466 51.85% 확률로 부정 리뷰입니다.

하도좋다고 광고로나오고 할인도하길래못이긴척 구매해봤습니다역시나 그냥그래요
1/1 [==============================] - 0s 22ms/step
467 91.33% 확률로 긍정 리뷰입니다.

파우더리 썬스틱 여러가지 테스트 해보고 이제품이 젤 괜찮아서 구매했어요
1/1 [==============================] - 0s 37ms/step
468 52.38% 확률로 부정 리뷰입니다.

넓어서 슥슥 펴바르기 편하고백탁현상 눈시림 없어서 좋아요 끈적임이나 번들거림도 없어서남편도 너무 만족해서 하나 더 구입했어요
1/1 [==============================] - 0s 45ms/step
469 56.41% 확률로 부정 리뷰입니다.

파우더리 제품 테스트 해봤는데 이제품이 제일 마음에 들어서 샀어요
1/1 [==============================] - 0s 41ms/step
470 52.38% 확률로 부정 리뷰입니다.

원래사용하던 브랜드인데 신상나왔길래 사보았습니다넓은 면적이라서 슥슥 금방얼굴에 바르고 나갈수있음 예민한피부에도 전혀 따갑지않고 스틱형이라 크림처럼 흘러내려서 눈 시릴일도없음 손에안묻으니 편해용
2/2 [==============================] - 0s 5ms/step
471 88.00% 확률로 부정 리뷰입니다.

휴대하고 다니면서 자외서 차단하기 좋은 제품입니다끈적이지도 않고 발림성도 좋은거같아요 자외선 차단
1/1 [==============================] - 0s 23ms/step
472 50.

2/2 [==============================] - 0s 17ms/step
493 52.41% 확률로 긍정 리뷰입니다.

골프를 시작한뒤 햇살이 따가운 봄이 시작되어 기미가 넘 걱정되었는데썬크림이 별도로 준비되어있어 패치를 붙이기 전에 썬크림 가득 피부보호할 곳곳에 바르고 그위 화장을 하니 밀리지도 않고 굿굿 필드나가서 패치를 붙이니 얼굴에 있는잡티 기미와 주근깨가 딱 가려져서 신경안쓰고 나이스샷 하고 왔네요 이런걸 왜 쓰는지 이유를 몰랐는데 써보니 확실히 기미차단에 효과가 있네요 앞으로 필드엔 필수품이 되었네요 쿠팡체험단 이벤트로 상품을 무료로 제공 받아 작성한 구매 후기입니다
3/3 [==============================] - 0s 10ms/step
494 52.12% 확률로 긍정 리뷰입니다.

인라인 수업을 받는데 햇빛이강해 잡티가올라오고 붉어지더라구여 그래서 구매해봤는데 썬크림도 같이 있어 착하네오 면적도 생각보다 넓어서 좋네요
1/1 [==============================] - 0s 31ms/step
495 54.82% 확률로 부정 리뷰입니다.

매우 추천합니다 붙이고 햇볕보고 들어오면 오히려 기미도 줄고 탄력까지 붙어있요 일단 최소량을 사서 써 보시고 대량구매 권해드립니다
1/1 [==============================] - 0s 60ms/step
496 86.82% 확률로 긍정 리뷰입니다.

우아 넓어서 좋아요 날개모양으로 귀엽구 피부보호가 된다고하니 기미 주근깨 생길 틈 없이 관리하기 또 선크림 양도 많아서 얼굴 목 넉넉하게 바르고도 남아서 다리에도 발라줬어요전부터 구매를 고민하던 제품인데 만족합니다
2/2 [==============================] - 0s 0s/step
497 61.47% 확률로 부정 리뷰입니다.

선물용으로 구매했어요위에 선크림이 함께 있어서굉장히 맘에들어 하시네요
1/1 [==============================] - 0s 

2/2 [==============================] - 0s 13ms/step
533 69.36% 확률로 부정 리뷰입니다.

시어머님께서 조명이 쎈곳에서 일하시는데 기미 낀다고 옆에 친구분이 쓰시는데 어떻게 사는지 모르겠다고 하셔서 대신 구매해드렸어요 친구분이 골프치실때 써보고 좋아서 사셨던것 같더라구요
2/2 [==============================] - 0s 16ms/step
534 53.85% 확률로 긍정 리뷰입니다.

베트남 여행시 기미 올라올까 싶어 급히 사가지고 갔는데 좋아요 들뜸이 생겨서 한번씩 눌러줘야 했지만 만족합니다 단 땀이 너무 많으면 끝부터 말리면서 계속 떨어져요 저는 땀이 별로 없는편이라 가족들중에 제가 제일 접착력 좋게 잘붙어 있었어요 얼굴땀 유독 많으신 분들은 비추해요
2/2 [==============================] - 0s 17ms/step
535 56.05% 확률로 긍정 리뷰입니다.

코로나 터지면서 화장을 안하게 되었는데 최근 거울을 들여다 보다 화들짝 놀랐네요 얼굴에기미가 정말 최악이네요 마스크 써서 햇빛에 덜 노출될 줄 착각하고 방심했습니다 이제부터라도외출할 때 신경 좀 쓰려고 구매하였습니다 착용감은 좋습니다
2/2 [==============================] - 0s 0s/step
536 82.89% 확률로 긍정 리뷰입니다.

여름철 외부활동이 많아서 구입했던건데요런 패치를 처음 사봤는데 괜찮았어요 드라마틱한 효과는 모르겠지만쿨링효과 있고 여름내 잘 사용했어요 선크림 바르고 붙이면 밀착효과가 떨어져요 패치 먼저 붙이고 선크림 발라야해요자극적이지 않아서 좋아요
2/2 [==============================] - 0s 18ms/step
537 50.22% 확률로 긍정 리뷰입니다.

골프치러 갈때 어머니가 필요하다고 해서 구매했어요안하는 것보다 얼굴도 덜 타고 더위도 덜하다고 해요 여름에 필수품인 것 같아요
1/1 [==================

...

2/2 [==============================] - 0s 0s/step
3070 53.28% 확률로 부정 리뷰입니다.

상품명 닥터지 레드 블레미쉬 시카 수딩 토너 200ml 에센스 플루이드 150ml 세트용량 토너 200ml 플루이드 150ml저는 닥터지 토너는 전부터 계속 사용중이였어요 닥터지는 유명하니까 모르시는 분들은 많이 없을거 같아요 토너는 향도 부담스럽지않고 수분기가 촉촉해서 금방 흡수가 되어 시간이 조금 지나면 겉 피부가 살짝 건조해져요 전 그래서 토너를 바르고 바로 에센스나 로션을 발라줘요 깔끔해져 너무 좋아요 플루이드는 묽어서 끈적임없이 잘 발려요 금방 촉촉해지고 흡수력도 좋아요 플루이드 바르고 로션바르시는게 좋을거같아요 닥터지는 성분이 순해서 저는 아직까지 트러블이 난적이 한번도 없어요 그리고 향도 쎄지않고 순해서 부담없이 사용하기 좋아요 쿠팡 체험단으로 선정되어 제품을 사용했지만 사용해보고 찐후기를 남긴거입니다 쿠팡체험단 이벤트로 상품을 무료로 제공 받아 작성한 구매 후기입니다
5/5 [==============================] - 0s 8ms/step
3071 52.99% 확률로 긍정 리뷰입니다.

시카토너 시카로션 스킨로션 보습끝판왕 닥터지레드 미백 주름개선 수분진정 점자표시굿 건강한제주병풀 병풀바이옴으로 촉촉하게 민감피부 시카진정 마침 스킨 로션이 거의다 떨어졌는데 요상품이 똭 보이더라구요 닥터지 레드 블레미쉬 시카토너 로션 스페셜 세트 스킨은 액상형으로 촉촉하고 쫙쫙 흡수되서 좋습니다 제가 겨울철 비염으로 향이 나는지는 모르겠지만 안좋은 냄세는 잘맡는편임 딱히 거슬릴만한 냄세는 없는것 같아용 저는 속수분이 부족하고 부분지성이라 복합성 피부로 볼수 있는데요배송 오자마자 테스트해본 결과 유수분 밸런스도 잡아준다는데 일단 촉촉하고 미끌거리지않고 잘흡수되서 만족합니다 유분많아서 미끌거리고 겉도는건 기름만 많아지는편이라 함께온 로션은 펌프식으로 나오는데 제가 얼큰이라 두번정도 짜서쓰면 딱인것 같구요 흡수잘되고

9/9 [==============================] - 0s 5ms/step
3080 55.00% 확률로 부정 리뷰입니다.

상품명 닥터지 레드 블레미쉬 시카 수딩 토너 200ml 에센스 플루이드 150ml 세트 사용기한 수딩 토너 2025 04 14 에센스 플루이드 2025 04 18 제조국 한국 제품설명 닥터지 레드 블레미쉬 시카 수딩 토너 생병풀 성분과 병풀 바이옴 이 민감해진 피부를 순하게 진정시켜주고 촉촉하게 수분을 채워주는 비건 시카 토너입니다 병풀 바이옴 생균이 아닌 용해물 혹은 사균체를 사용한 원료 외부 자극에 의한 피부장벽 개선 및 인체적용시험 완료 각질층 10층까지의 수분 보습력에 대한 인체적용시험 완료 민감성 피부 사용 적합 인체적용시험 완료 피부 자극 테스트 완료 닥터지 레드 블레미쉬 밸런싱 에센스 플루이드 끈적임 없이 흡수 빠른 플루이드 포뮬라에 농축시킨 에센스 성분이 유수분 밸런스를 맞춰주고 5 CICA complex 와 병풀 바이옴이 피부장벽을 건강하게 케어해주는 약산성 수분 진정 플루이드입니다 5 CICA complex 5가지 시카 핵심 성분인 마데카소사이드  아시아틱애씨드 아시아티코사이드 마데카식애씨드  병풀 추출물로 이루어진 복합 성분 미백 주름개선 2중 기능성 화장품 여드름성 피부 사용 적합 인체적용시험 완료 2중 보습 개선 평가시험 완료 민감성 피부 사용 적합 인체적용시험 완료 피부 자극 테스트 완료 사용법 닥터지 레드 블레미쉬 시카 수딩 토너 스킨케어 첫 단계에서 적당량을 덜어 부드럽게 펴바르고 가볍게 두드리듯 흡수시켜줍니다 스페셜 케어가 필요할 때에는 2 3번 레이어링 하여 흡수시켜주세요 닥터지 레드 블레미쉬 밸런싱 에센스 플루이드 적당략을 덜어 얼굴에 골고루 펴 바른 후 가볍게 두드려 흡수시켜 줍니다 사용후기 저는 토너 같은 경우 물 같이 묽은 느낌이여서 솜에 적당량을 덜어 사용했어요 끈적임이 없고 부드럽게 흡수되어 가볍지만 또 건조한 건 아니라서 좋아요 에센스 플루이드 같은 경우에도 되게 무겁지 않고 가

5/5 [==============================] - 0s 4ms/step
3089 51.08% 확률로 부정 리뷰입니다.

닥터지 레드 블레미쉬 시카 수딩 토너 200ml 에센스 플루이드 150ml 세트 집에 있으니 완전 든든해요 병풀 바이옴 성분으로 민감 피부를 편안하게 진정시켜주는 시카케어 제품인데요 민감한 피부를 가지신 분들이 사용하기 딱 좋은 제품 같아요 제가 피부가 예민하기도 하고 가끔 뾰루지가 나는데 그럴때마다 병풀 추출물이 함유된 제품을 쓰면 효과가 좋더라구요 이번 기회에 닥터지 토너 에센스 세트로 집에 구비해두니 피부 걱정 없네요 시카 수딩 토너는 묽은 토너라서 닦토로 사용해봤는데 진짜 좋더라구요 밀리지 않고 촉촉하게 피부에 수분 충전이 가능해요 그리고 에센스 플루이드는 유분을 걷고 수분을 채워주는 피부 밸런싱 케어가 가능하다고 하더라구요 피부에 기본적으로 유분이 좀 많은 편이라 유분 없애고 수분으로 가득 채워주고 싶더라구요 일단 저는 개인적으로 끈적이는 제형은 정말 싫은데 전혀 끈적이지 않아요 그리고 기초 제품이다 보니 흡수가 잘되야 하고 메이크업을 했을때 밀리지 않는게 제일 좋은데 진짜 안밀리더라구요 수분은 바로바로 제 피부 속으로 그리고 예민한 피부인 제가 사용해도 따갑거나 이런 느낌은 전혀 없었어요 용량도 대용량이고 대만족합니다 쿠팡체험단 이벤트로 상품을 무료로 제공 받아 작성한 구매 후기입니다
7/7 [==============================] - 0s 7ms/step
3090 51.08% 확률로 부정 리뷰입니다.

평소에 px에서 파는 닥터지 수딩크림을 쓰다가닥터지는 시카라인으로 유명해서 로션 스킨도 주문해봤어요 우선 향이 강하지 않아서 순딩순딩한 제품이구나 싶었습니다 스킨은 얼굴에 바로 잘 흡수가 되어서 유분기가 올라오거나 하지 않아서 좋았습니다 로션은 얼굴에 올리고 난 후 시원한 쿨링감이 느껴져서 좋았어요 강하지는 않음 5일정도 써봤는데 뾰루지 2 3개 정도 올라왔었는데 잠을 잘 자서 그런건지 제품이 

1/1 [==============================] - 0s 35ms/step
3114 54.01% 확률로 긍정 리뷰입니다.

잘맞는지모르겠습니다
1/1 [==============================] - 0s 34ms/step
3115 61.61% 확률로 부정 리뷰입니다.

기초화장품 바꾸려고 샀는데 바르고 오분정도 있으니 피부가 가렵고 따갑네요 거울을 보니 피부가 다 뒤집어져 그 몇분 사이에 여드름 뾰루지들이 올라왔어요 한번 썼는데 버려야하네요 돈도 버리고 피부도 버리고 너무 속상합니다 저는 수분부족민감형피부입니다 저랑 같은 피부타입의 예민한 피부를 가지신 분들은 성분 보시고 사세요
2/2 [==============================] - 0s 18ms/step
3116 65.44% 확률로 긍정 리뷰입니다.

제가 건조한피부는 아닌데 써보니 촉촉한느낌은 그닥 수딩젤 수분크림 써보고 너무좋아서 ahc 랑고민하다가구입했는데 실패내여 가격이 저렴한편인데 상품에 비해 아까워요 뜯어서 반품할수도 없고 완전실망요 완전 순수한제품입니다
2/2 [==============================] - 0s 18ms/step
3117 53.60% 확률로 긍정 리뷰입니다.

주문 23년 4월 18일수령 23년 4월 20일유통기한 26년 3월 5일까지쿠팡 비닐에 본품상자만 담겨져서 배송되었습니다 따로 뽁뽁이등으로 감싸져있거나한건 없었어요 상자 사이드가 찌그러져왔고 뭔가 노란색이 묻어있고 먼지가 많아보였지만다행히 파손 등 이상은 없었습니다 스킨과 에센스 용량도 넉넉한 편이고에센스는 펌프타입이라 사용도 편리하네요 향은 약간 나는듯한데 무향에 가까워 거부감없고물제형에 가깝습니다 가벼우면서도 맑은 느낌? 드는데 촉촉한거 같아요 트러블이 은근 잘나는데 병풀추출물 성분이 함유되어있으니 진정과 트러블 예방에 도움될듯 합니다 남동생이 닥터지만 사용하던데 왜 그런지 알듯해요쿠팡체험단으로 사용하였으나리뷰를 보고 구매에 참고하실 분들을 위해최대한 개인적

AssertionError: phrase input should be string, not <class 'float'>

In [58]:
positive

[[0, 0.8691211938858032],
 [1, 0.7572132349014282],
 [2, 0.5240933299064636],
 [4, 0.5240933299064636],
 [5, 0.5281047224998474],
 [6, 0.5241168737411499],
 [9, 0.5124542713165283],
 [12, 0.5241168737411499],
 [13, 0.9133251309394836],
 [17, 0.6330621242523193],
 [18, 0.5241168737411499],
 [20, 0.5241168737411499],
 [22, 0.7685761451721191],
 [23, 0.5059183239936829],
 [24, 0.5255030393600464],
 [25, 0.6476082801818848],
 [26, 0.6908936500549316],
 [27, 0.6505851149559021],
 [29, 0.5359945297241211],
 [31, 0.7444575428962708],
 [33, 0.5611952543258667],
 [34, 0.8207664489746094],
 [36, 0.6701986789703369],
 [40, 0.5657661557197571],
 [42, 0.5421618819236755],
 [43, 0.8527746200561523],
 [45, 0.7374867796897888],
 [48, 0.5241168737411499],
 [49, 0.6635440587997437],
 [51, 0.5241168737411499],
 [52, 0.5890361666679382],
 [54, 0.5485531687736511],
 [56, 0.9133251309394836],
 [57, 0.5255030393600464],
 [58, 0.5400121212005615],
 [59, 0.5448511838912964],
 [60, 0.744457483291626],
 [66, 0.5

In [73]:
len(senti_prob)

301

In [75]:
df = pd.read_excel("./csv/NoEvinue.xlsx")
df = df.iloc[300:601]
df['sentiment_prob'] = senti_prob
# df

In [62]:
df.to_excel("noEvinue_senti.xlsx")

In [80]:
df = pd.read_excel("./csv/Evinue.xlsx")
df = df.iloc[300:600]
df['sentiment_prob'] = senti_prob

In [82]:
df.to_excel("Evinue_senti2.xlsx")